In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
mordred = pd.read_csv("../data/all_f_main_dataset_mordred_V2.csv", index_col=0)
zeo = pd.read_csv("../data/main_dataset_zeo_V2.csv", index_col=0)
cif = pd.read_csv("../data/main_dataset_cif_property_V2.csv", index_col=0)



In [4]:
cif

,a,b,c,alpha,beta,gamma,volume,sg_number
initial MOF-5-C7 2040922.cif,12.852600,12.852600,12.852600,90.000000,90.000000,90.000000,2123.112341,221.0
to MOF-5(C7)_desolvated phase 2040923.cif,12.786337,12.786337,12.786337,84.507765,84.507765,84.507765,2063.376733,148.0
from Cu(I)-MOF-1 2084812.cif,13.393384,13.393384,13.393384,106.730376,106.730376,115.104051,1835.938608,88.0
from Cu(I)Cu(II)-MOF-2 2084857.cif,13.124566,13.124566,13.124566,107.741062,107.741062,112.989882,1735.380592,88.0
to Cu(II)-MOF-3 1524653.cif,10.044000,13.671000,13.671000,90.000000,90.000000,90.000000,2081.642704,60.0
...,...,...,...,...,...,...,...,...
1b_40 bar(R).cif,8.249900,18.659300,18.659300,90.000000,90.000000,90.000000,2872.363364,90.0
from 1a.cif,8.047300,18.855000,18.855000,90.000000,90.000000,90.000000,2860.903871,90.0
to 1b.cif,7.797900,16.850000,16.850000,90.000000,90.000000,90.000000,2214.656236,18.0
as-synthesized SNU-80a (initial) 842625.cif,12.223300,24.807600,24.807600,90.000000,90.000000,90.000000,7522.426833,134.0


In [3]:
mordred.shape, zeo.shape, cif.shape

((151, 1826), (162, 3), (165, 8))

In [4]:
def make_mordred():
    from mordred import Calculator, descriptors
    from rdkit import Chem
    mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
    mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
    linkers = [mofid.smiles_linkers[i].replace("'", "")[1:-1].split(", ") for i in range(mofid.__len__())]
    mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
    calc = Calculator(descriptors, ignore_3D=False)
    def f(mof):
        try: return calc.pandas(mof)
        except TypeError:
            return None
    dfs = [f(mof) for mof in mols]
    data_mordred = pd.DataFrame(columns=dfs[0].columns)

    for i, filename in enumerate(mofid.index):
        try:
            if linkers[i] != [""]:
                data_mordred.loc[filename] = dfs[i].mean()
        except AttributeError:
            print(f"{filename:_^20}")
    data_mordred = data_mordred.set_index(data_mordred.index.map(lambda name: name.replace(" ", "")))
    #data_mordred.to_csv("../data/all_f_main_dataset_mordred_V2.csv")
    return data_mordred



In [5]:
def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [6]:
elemental_descriptors = pd.read_csv("../data/elemental_descriptors.csv")

In [7]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid.index.map(lambda name: name.replace(" ", "").replace(".cif", "")), mofid["smiles_nodes"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with []
Error with []
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']


In [8]:
node_descriptors = node_descriptors.loc[node_descriptors["n_types_metals"] == 1.]


In [9]:
zeo.index = zeo.index.map(lambda s: s.replace(".res", ""))

In [10]:
data_pld_lcd = zeo[["num1", "num2"]].rename({"num1": "lcd", "num2": "pld"}, axis=1)

In [11]:
cif = cif.rename({ "sg_number": "spacegroupNumber"}, axis=1)

In [12]:
cif.index = cif.index.map(lambda name: name.replace(" ", "").replace(".cif", ""))

In [13]:
cif = cif.set_index(cif.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [14]:
node_descriptors = node_descriptors.set_index(node_descriptors.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [15]:
data_pld_lcd = data_pld_lcd.set_index(data_pld_lcd.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [16]:
mordred = mordred.set_index(mordred.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [17]:
data_pld_lcd.shape, cif.shape, mordred.shape, node_descriptors.shape

((162, 2), (165, 8), (151, 1826), (145, 8))

In [18]:
data_main = pd.concat([data_pld_lcd, cif, mordred, node_descriptors], axis=1)

In [19]:
data_main = data_main.loc[list({*list(data_pld_lcd.index)} & {*list(
    cif.index)} & {*list(mordred.index)} & {*list(node_descriptors.index)})]


In [20]:
data_main.shape

(138, 1844)

In [21]:
# data_main.to_csv("../data/main_no_marked.csv")

In [22]:
db_from_to = pd.read_csv("../data/DB_main_with_SG_for_EDA.csv", sep=';')

In [23]:
db_from_to.CIF_init = db_from_to.CIF_init.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))
db_from_to.CIF_final = db_from_to.CIF_final.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))


In [24]:
marked = pd.DataFrame(columns=["target"])

for ind in db_from_to.index:
    if db_from_to.loc[ind, "Stimuli"].find("solvent") != -1 or db_from_to.loc[ind, "Stimuli"].find("gas") != -1 or db_from_to.loc[ind, "Stimuli"].find("humidity") != -1:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [1]
        if db_from_to.loc[ind, "Reversible"] == "yes":
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [1]
        else:
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]
    else:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [0]
        marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]

In [25]:
from pteproc_model import PreprocessingModel
model = joblib.load("../qmof_datasets/scaler.pkl")

In [26]:
data_main.shape

(138, 1844)

In [27]:
main_dataset = data_main[model.cols]

In [28]:
marked_dataset = pd.concat([data_main, marked], axis=1)

In [29]:
marked_dataset = marked_dataset.loc[list({*list(main_dataset.index)} & {*list(marked.index)})]

In [30]:
marked_dataset.shape

(102, 1845)

In [31]:
target, counts = np.unique(marked_dataset.target, return_counts=True)


print(f"|{'target':^10}|{'counts':^10}|")
print(f"{'-':-^23}")
print(f"|{'yes':^10}|{counts[0]:^10}|")
print(f"|{'no':^10}|{counts[1]:^10}|")
print(f"total count:{marked_dataset.__len__()}")

|  target  |  counts  |
-----------------------
|   yes    |    62    |
|    no    |    40    |
total count:102


In [32]:
marked_dataset.shape

(102, 1845)

In [33]:
x, y = model.transform(marked_dataset.drop(['target'], axis=1)), marked_dataset['target']

In [34]:
x.shape, x.dropna().shape

((102, 1018), (102, 1018))

In [39]:
for i in x.index:
    print(i)

from4in61536823
from3in61515548
1to71031676
initialZn-1238860
2to2byimmercingfirstcompound1816056
to6cm6b05277_si_005
toCuII-MOF-31524653
initialMOFDMF@DUT-8Ni1989709
2to101045134
fromCuI-MOF-12084812
fromassynthesizedDUT-30Zn804770
to1UPC-600-373K2060465
to3fa1052173
2from2924308-Copy
to5cm6b05277_si_004
toic035111o_si_002
toCCDC_2065041_JUK-20-noh
from935302
toZn-1withDMF238859
toSTA-26Zr-Cchangedsymmetry1571656
fromic035111o_si_002
2from91045135
from31483716
toDUT-49Fe100K2014975
toic035111o_si_003
basedonCofromPnna709782
fromja500530y_si_003
fromCuICuII-MOF-22084857
toVMOP-+1590348
to1b
topart_4
to6cm6b05277_si_005-Copy
BasedonMntoR-3c
to922086
from975784
toDUT-49Zn100K2014973
fromDUT-49Fe296K1957603
fromas-flexMOF975746
fromDUT-49Zn296K890363
to1934735
toDUT-49Ni100K2014970
toCo-MOF1912233
toCAU13_-1GPa
fromDUT-49Mn296K1957604
to41483725
basedonMnfromPnna709780
initialSTA-26Zr1571655
toCCDC_2065039_JUK-20-chx
fromDUT-49Cd296K1957602
fromCo-MOF1912199
toMOF-74Zn1512654
from1in56369

In [36]:
x.to_csv("../main_datasets/dataset.csv")

In [37]:
y.to_csv("../main_datasets/target.csv")